# RMH Geospatial API Examples v1

**Date**: 05 DEC 2025  
**Author**: Robert and Geospatial Claude Legion

This notebook demonstrates all available API endpoints for DDH (Data Discovery Hub) integration.

## Endpoints Covered

### Health & Diagnostics
- `GET /api/health` - System health check

### Platform Layer (DDH Integration)
- `POST /api/platform/submit` - Generic submission (auto-detects data type)
- `POST /api/platform/raster` - Single raster submission
- `POST /api/platform/raster-collection` - Multiple raster submission
- `GET /api/platform/status/{request_id}` - Check request status

### Job Management
- `POST /api/jobs/submit/{job_type}` - Direct job submission
- `GET /api/jobs/status/{job_id}` - Check job status

### Database Queries
- `GET /api/dbadmin/jobs` - Query jobs
- `GET /api/dbadmin/tasks/{job_id}` - Query tasks for a job
- `GET /api/dbadmin/stats` - Database statistics

### STAC API
- `GET /api/collections` - List STAC collections
- `GET /api/collections/{collection_id}` - Get collection details
- `GET /api/collections/{collection_id}/items` - Get items in collection

### OGC Features API
- `GET /api/features/collections` - List vector collections
- `GET /api/features/collections/{collection}/items` - Query features

## Setup

In [ ]:
import requests
import json
from pprint import pprint

# =============================================================================
# CONFIGURATION - Azure Account-Specific Values
# =============================================================================
# Modify these values when deploying to a different Azure tenant

# Function App Base URL
FUNCTION_APP_NAME = "rmhazuregeoapi"
FUNCTION_APP_SUBDOMAIN = "a3dma3ctfdgngwf6"  # From Azure deployment
AZURE_REGION = "eastus-01"
BASE_URL = f"https://{FUNCTION_APP_NAME}-{FUNCTION_APP_SUBDOMAIN}.{AZURE_REGION}.azurewebsites.net"

# Storage Containers (Bronze tier - raw data)
BRONZE_VECTORS_CONTAINER = "bronze-vectors"
BRONZE_RASTERS_CONTAINER = "bronze-rasters"
BRONZE_MISC_CONTAINER = "bronze-misc"
BRONZE_LEGACY_CONTAINER = "rmhazuregeobronze"  # Legacy flat container

# Storage Containers (Silver tier - processed data)
SILVER_COGS_CONTAINER = "silver-cogs"
SILVER_VECTORS_CONTAINER = "silver-vectors"

# Database Schema
POSTGIS_SCHEMA = "geo"

# STAC Collection IDs
DEFAULT_COLLECTION_ID = "dev"          # Development/testing collection
VECTORS_COLLECTION_ID = "vectors"      # User-submitted vectors
COGS_COLLECTION_ID = "cogs"            # User-submitted COGs

# Access Levels
DEFAULT_ACCESS_LEVEL = "OUO"           # Options: public, OUO, restricted

# =============================================================================
# Helper Functions
# =============================================================================

def api_call(method, endpoint, data=None, params=None, timeout=30):
    """Make API call and return formatted response.
    
    Args:
        method: HTTP method (GET or POST)
        endpoint: API endpoint path
        data: Request body for POST requests
        params: Query parameters for GET requests
        timeout: Request timeout in seconds (default: 30, use 90 for /api/health)
    """
    url = f"{BASE_URL}{endpoint}"
    headers = {"Content-Type": "application/json"}
    
    print(f"\n{'='*60}")
    print(f"{method} {endpoint}")
    print(f"{'='*60}")
    
    if data:
        print(f"\nRequest Body:")
        print(json.dumps(data, indent=2))
    
    try:
        if method == "GET":
            response = requests.get(url, params=params, timeout=timeout)
        elif method == "POST":
            response = requests.post(url, json=data, headers=headers, timeout=timeout)
        else:
            raise ValueError(f"Unsupported method: {method}")
        
        print(f"\nStatus: {response.status_code}")
        
        try:
            result = response.json()
            print(f"\nResponse:")
            print(json.dumps(result, indent=2, default=str))
            return result
        except:
            print(f"\nResponse (text): {response.text[:500]}")
            return response.text
            
    except requests.exceptions.Timeout:
        print(f"\n❌ Request timed out (timeout={timeout}s)")
        return None
    except Exception as e:
        print(f"\n❌ Error: {e}")
        return None

# Display configuration
print("=" * 60)
print("API CONFIGURATION")
print("=" * 60)
print(f"Base URL: {BASE_URL}")
print(f"Function App: {FUNCTION_APP_NAME}")
print(f"Bronze Containers: {BRONZE_VECTORS_CONTAINER}, {BRONZE_RASTERS_CONTAINER}")
print(f"Bronze Legacy: {BRONZE_LEGACY_CONTAINER}")
print(f"Silver Containers: {SILVER_COGS_CONTAINER}")
print(f"PostGIS Schema: {POSTGIS_SCHEMA}")
print(f"Default Collection: {DEFAULT_COLLECTION_ID}")
print("=" * 60)

---
# 1. Health & Diagnostics

## 1.1 Health Check

Comprehensive health check endpoint. Returns system status and component health.

**Note**: This endpoint takes ~60 seconds to complete due to:
- Database connectivity and query tests
- Service Bus queue checks (4 queues)
- PgSTAC version and table verification
- Schema inspection across all schemas
- PostgreSQL function tests

Use `timeout=90` when calling this endpoint.

In [ ]:
# Health Check
# NOTE: Health endpoint takes ~60 seconds due to comprehensive checks
# (database, Service Bus queues, PgSTAC, schema inspection)
result = api_call("GET", "/api/health",timeout =90)

---
# 2. Platform Layer (DDH Integration)

The Platform layer is the primary integration point for DDH. It translates DDH requests to CoreMachine jobs.

**Key Concepts:**
- `dataset_id`: Top-level organizational unit (e.g., "aerial-imagery-2024")
- `resource_id`: Second-level identifier (e.g., "site-alpha")
- `version_id`: Version control (e.g., "v1.0")
- `request_id`: Generated from SHA256(dataset_id + resource_id + version_id) - idempotent

## 2.1 Submit Vector via Platform

Submit a vector file (GeoJSON, Shapefile, GeoPackage, etc.) for ingestion into PostGIS.

In [ ]:
# Submit Vector via Platform (generic endpoint)
# ✅ TESTED: 8.geojson → 3,879 features loaded to geo.test_vectors_geojson_8_v1
vector_request = {
    "dataset_id": "test-vectors",
    "resource_id": "geojson-8",
    "version_id": "v1",
    "container_name": BRONZE_LEGACY_CONTAINER,
    "file_name": "8.geojson",
    "service_name": "Test GeoJSON 8"
}

result = api_call("POST", "/api/platform/submit", vector_request)

## 2.2 Submit Single Raster via Dedicated Endpoint

Use `/api/platform/raster` for single raster files. Platform automatically handles size-based routing:
- Files ≤ 1GB → `process_raster_v2`
- Files > 1GB → `process_large_raster_v2` (automatic fallback)

### 2.2a Small Raster Example (< 1GB)

In [ ]:
# Submit Single Raster (dedicated endpoint)
# ✅ TESTED: dctest.tif (25.8 MB) → COG 127.57 MB, STAC item dctest-analysis-v2
single_raster_request = {
    "dataset_id": "dctest",
    "resource_id": "analysis",
    "version_id": "v2",
    "container_name": BRONZE_LEGACY_CONTAINER,
    "file_name": "dctest.tif",
    "service_name": "DC Test Analysis v2"
}

result = api_call("POST", "/api/platform/raster", single_raster_request)

### 2.2b Large Raster Example (> 1GB, automatic fallback)

When a large raster is submitted to `/api/platform/raster`, it automatically routes to `process_large_raster_v2`.

In [ ]:
# Submit Large Raster (same endpoint, automatic fallback to process_large_raster_v2)
# ✅ TESTED: 2021_Luang_Prabang_DTM.tif (986 MB) → routed to process_large_raster_v2
large_raster_request = {
    "dataset_id": "laos-dtm",
    "resource_id": "luang-prabang",
    "version_id": "2021",
    "container_name": BRONZE_LEGACY_CONTAINER,
    "file_name": "2021_Luang_Prabang_DTM.tif",
    "service_name": "Luang Prabang DTM 2021"
}

result = api_call("POST", "/api/platform/raster", large_raster_request)

## 2.3 Submit Raster Collection via Dedicated Endpoint

Use `/api/platform/raster-collection` for multiple raster files that should be processed as a collection with MosaicJSON.

In [ ]:
# Submit Raster Collection (dedicated endpoint)
# ✅ TESTED: namangan/ 4 tiles → 1669.75 MB COGs, MosaicJSON, search_id for TiTiler
collection_request = {
    "dataset_id": "namangan-imagery",
    "resource_id": "aug2019",
    "version_id": "v1",
    "container_name": BRONZE_LEGACY_CONTAINER,
    "file_name": [  # Must be list with at least 2 files
        "namangan/namangan14aug2019_R1C1cog.tif",
        "namangan/namangan14aug2019_R1C2cog.tif",
        "namangan/namangan14aug2019_R2C1cog.tif",
        "namangan/namangan14aug2019_R2C2cog.tif"
    ],
    "service_name": "Namangan Imagery August 2019"
}

result = api_call("POST", "/api/platform/raster-collection", collection_request)

## 2.4 Check Platform Request Status

Poll the status of a Platform request using the `request_id` returned from submission.

In [ ]:
# Check Platform Request Status
# Replace with actual request_id from a submission
request_id = "c3945f0af8f4f135c1012324b6e38394"

if request_id != "YOUR_REQUEST_ID_HERE":
    result = api_call("GET", f"/api/platform/status/{request_id}")
else:
    print("⚠️ Replace 'YOUR_REQUEST_ID_HERE' with an actual request_id from a submission above")

## 2.5 List All Platform Requests

List recent Platform requests with optional filtering.

In [ ]:
# List Platform Requests
result = api_call("GET", "/api/platform/status", params={"limit": 10})

---
# 3. Direct Job Submission

For advanced use cases, jobs can be submitted directly to CoreMachine, bypassing the Platform layer.

## 3.1 Hello World Job (Test)

Simple test job to verify the system is working.

In [ ]:
# Hello World Job
hello_params = {
    "message": "Hello from API notebook!",
    "count": 3
}

result = api_call("POST", "/api/jobs/submit/hello_world", hello_params)

## 3.2 Process Vector Job (Direct)

Direct vector processing job submission.

In [ ]:
# Process Vector Job (direct submission)
process_vector_params = {
    "blob_name": "example.geojson",
    "container_name": BRONZE_VECTORS_CONTAINER,
    "table_name": "example_direct_vector",
    "schema": POSTGIS_SCHEMA,
    "file_extension": "geojson"
}

result = api_call("POST", "/api/jobs/submit/process_vector", process_vector_params)

## 3.3 Process Raster V2 Job (Direct)

Direct single raster processing.

In [ ]:
# Process Raster V2 Job (direct submission)
process_raster_params = {
    "blob_name": "example.tif",
    "container_name": BRONZE_RASTERS_CONTAINER,
    "collection_id": DEFAULT_COLLECTION_ID,
    "output_folder": "test/direct-raster"
}

result = api_call("POST", "/api/jobs/submit/process_raster_v2", process_raster_params)

## 3.4 Process Raster Collection V2 Job (Direct)

Direct raster collection processing with MosaicJSON.

In [ ]:
# Process Raster Collection V2 Job (direct submission)
collection_params = {
    "blob_list": ["tile1.tif", "tile2.tif", "tile3.tif"],
    "container_name": BRONZE_RASTERS_CONTAINER,
    "collection_id": DEFAULT_COLLECTION_ID,
    "output_folder": "test/direct-collection"
}

result = api_call("POST", "/api/jobs/submit/process_raster_collection_v2", collection_params)

## 3.5 Process Large Raster V2 Job (Direct)

Direct large raster processing (files > 1GB).

In [ ]:
# Process Large Raster V2 Job (direct submission)
large_raster_params = {
    "blob_name": "large_raster.tif",
    "container_name": BRONZE_RASTERS_CONTAINER,
    "collection_id": DEFAULT_COLLECTION_ID,
    "output_folder": "test/large-raster"
}

result = api_call("POST", "/api/jobs/submit/process_large_raster_v2", large_raster_params)

## 3.6 Check Job Status

Check the status of a CoreMachine job.

In [ ]:
# Check Job Status
# Replace with actual job_id from a submission
job_id = "YOUR_JOB_ID_HERE"

if job_id != "YOUR_JOB_ID_HERE":
    result = api_call("GET", f"/api/jobs/status/{job_id}")
else:
    print("⚠️ Replace 'YOUR_JOB_ID_HERE' with an actual job_id from a submission above")

---
# 4. Database Query Endpoints

Query jobs and tasks directly from the database. Useful for monitoring and debugging.

## 4.1 Query Jobs

Query jobs with optional filtering by status, job_type, or time range.

In [ ]:
# Query recent jobs
result = api_call("GET", "/api/dbadmin/jobs", params={"limit": 10})

In [ ]:
# Query jobs by status
result = api_call("GET", "/api/dbadmin/jobs", params={"status": "completed", "limit": 5})

In [ ]:
# Query jobs by type
result = api_call("GET", "/api/dbadmin/jobs", params={"job_type": "hello_world", "limit": 5})

## 4.2 Query Tasks for a Job

Get all tasks associated with a specific job.

In [ ]:
# Query tasks for a specific job
job_id = "YOUR_JOB_ID_HERE"

if job_id != "YOUR_JOB_ID_HERE":
    result = api_call("GET", f"/api/dbadmin/tasks/{job_id}")
else:
    print("⚠️ Replace 'YOUR_JOB_ID_HERE' with an actual job_id")

## 4.3 Database Statistics

Get overall database statistics including job/task counts and schema info.

In [ ]:
# Database Statistics
result = api_call("GET", "/api/dbadmin/stats")

---
# 5. STAC API

SpatioTemporal Asset Catalog (STAC) API for discovering and accessing geospatial data.

## 5.1 List STAC Collections

In [ ]:
# List all STAC collections
result = api_call("GET", "/api/collections")

## 5.2 Get Collection Details

In [ ]:
# Get specific collection details
result = api_call("GET", f"/api/collections/{DEFAULT_COLLECTION_ID}")

## 5.3 List Items in Collection

In [ ]:
# List items in a collection
result = api_call("GET", f"/api/collections/{DEFAULT_COLLECTION_ID}/items", params={"limit": 10})

## 5.4 STAC Search

In [ ]:
# STAC Search with bbox
search_params = {
    "collections": [DEFAULT_COLLECTION_ID],
    "limit": 10
}
result = api_call("POST", "/api/search", search_params)

---
# 6. OGC Features API

OGC API - Features for accessing vector data stored in PostGIS.

## 6.1 List Feature Collections

In [ ]:
# List all OGC feature collections (PostGIS tables)
result = api_call("GET", "/api/features/collections")

## 6.2 Get Collection Metadata

In [ ]:
# Get specific collection metadata
# Replace with an actual table name from the list above
collection_name = "YOUR_TABLE_NAME"

if collection_name != "YOUR_TABLE_NAME":
    result = api_call("GET", f"/api/features/collections/{collection_name}")
else:
    print("⚠️ Replace 'YOUR_TABLE_NAME' with an actual table name from the collections list")

## 6.3 Query Features

In [ ]:
# Query features from a collection
collection_name = "YOUR_TABLE_NAME"

if collection_name != "YOUR_TABLE_NAME":
    result = api_call("GET", f"/api/features/collections/{collection_name}/items", 
                      params={"limit": 10})
else:
    print("⚠️ Replace 'YOUR_TABLE_NAME' with an actual table name")

## 6.4 Spatial Query with Bounding Box

In [ ]:
# Query features with bounding box (minx, miny, maxx, maxy in EPSG:4326)
collection_name = "YOUR_TABLE_NAME"
bbox = "-180,-90,180,90"  # World extent

if collection_name != "YOUR_TABLE_NAME":
    result = api_call("GET", f"/api/features/collections/{collection_name}/items",
                      params={"bbox": bbox, "limit": 10})
else:
    print("⚠️ Replace 'YOUR_TABLE_NAME' with an actual table name")

---
# 7. Container Operations

## 7.1 List Container Contents

In [ ]:
# List Container Contents Job
list_params = {
    "container_name": BRONZE_RASTERS_CONTAINER,
    "prefix": "",
    "max_results": 100
}

result = api_call("POST", "/api/jobs/submit/list_container_contents", list_params)

## 7.2 Summarize Container

In [ ]:
# Summarize Container Job
summary_params = {
    "container_name": BRONZE_RASTERS_CONTAINER
}

result = api_call("POST", "/api/jobs/submit/summarize_container", summary_params)

## 7.3 Geospatial Inventory

In [ ]:
# Inventory Container for Geospatial Files
inventory_params = {
    "container_name": BRONZE_RASTERS_CONTAINER,
    "prefix": ""
}

result = api_call("POST", "/api/jobs/submit/inventory_container_geospatial", inventory_params)

---
# 8. H3 Grid Operations

## 8.1 Create H3 Base Grid

In [ ]:
# Create H3 Base Grid
h3_params = {
    "resolution": 4,
    "table_name": "h3_res4_test"
}

result = api_call("POST", "/api/jobs/submit/create_h3_base", h3_params)

---
# 9. STAC Catalog Operations

## 9.1 Catalog Container to STAC

In [ ]:
# Catalog Container Contents to STAC
catalog_params = {
    "container_name": SILVER_COGS_CONTAINER,
    "collection_id": DEFAULT_COLLECTION_ID,
    "prefix": ""
}

result = api_call("POST", "/api/jobs/submit/stac_catalog_container", catalog_params)

## 9.2 Catalog Vectors to STAC

In [ ]:
# Catalog PostGIS Vectors to STAC
vector_catalog_params = {
    "schema": POSTGIS_SCHEMA,
    "collection_id": VECTORS_COLLECTION_ID
}

result = api_call("POST", "/api/jobs/submit/stac_catalog_vectors", vector_catalog_params)

---
# 10. Available Job Types Reference

Complete list of available job types in the system.

In [ ]:
# Available Job Types
job_types = {
    "Production Workflows": [
        "hello_world",
        "summarize_container",
        "list_container_contents",
        "stac_catalog_container",
        "stac_catalog_vectors",
        "validate_raster_job",
        "generate_h3_level4",
        "create_h3_base",
        "bootstrap_h3_land_grid_pyramid",
    ],
    "Vector & Raster ETL": [
        "process_vector",
        "process_raster_v2",
        "process_raster_collection_v2",
        "process_large_raster_v2",
    ],
    "Fathom ETL": [
        "process_fathom_stack",
        "process_fathom_merge",
    ],
    "Container Analysis": [
        "inventory_container_geospatial",
        "inventory_fathom_container",
    ],
    "Test/Diagnostic": [
        "list_container_contents_diamond",
    ]
}

print("Available Job Types:")
print("=" * 50)
for category, jobs in job_types.items():
    print(f"\n{category}:")
    for job in jobs:
        print(f"  - {job}")

---
# 11. Platform Endpoints Reference

Summary of Platform layer endpoints for DDH integration.

In [ ]:
platform_endpoints = {
    "Submission Endpoints": {
        "POST /api/platform/submit": "Generic submission (auto-detects data type)",
        "POST /api/platform/raster": "Single raster (size-based routing)",
        "POST /api/platform/raster-collection": "Multiple rasters (MosaicJSON)",
    },
    "Status Endpoints": {
        "GET /api/platform/status/{request_id}": "Get specific request status",
        "GET /api/platform/status": "List all requests (with filters)",
    },
    "Required Fields (All Submissions)": [
        "dataset_id",
        "resource_id", 
        "version_id",
        "container_name",
        "file_name",
    ],
    "Optional Fields": [
        "service_name",
        "description",
        "access_level (default: OUO)",
        "processing_options",
    ]
}

print("Platform Layer Endpoints:")
print("=" * 60)
for section, content in platform_endpoints.items():
    print(f"\n{section}:")
    if isinstance(content, dict):
        for endpoint, desc in content.items():
            print(f"  {endpoint}")
            print(f"    → {desc}")
    else:
        for item in content:
            print(f"  - {item}")